In [9]:
import nbformat as nbf
from nbconvert import PDFExporter
import random
import string

# tweaked the prettyprint functionality in Python to stop it from sorting the dictionary when it prints
import pprint_jhs


In [10]:
nb = nbf.v4.new_notebook()

titlePDF  = 'Intelligent Systems 1'
datePDF   = 'Lab 2'
authorPDF = 'Dr James Stovold'

Your unit tests / random question generation code goes in the code chunk below. In this example I am recursively generating random trees for the students to run breadth-first search on, so I've also implemented a version of BFS to test against. 

In [11]:
def bfs(visited, graph, node):
    queue = []
    visited.append(node)
    queue.append(node)

    while queue:
        s = queue.pop(0) 
        print (s, end = " ") 

        for neighbour in graph[s]:
            if neighbour not in visited:
                visited.append(neighbour)
                queue.append(neighbour)
    
    return visited

def getRandomName(length = 4, exclude_list = []):
    # return a random text string of length `length`
    thisName = ''.join([random.choice(string.ascii_lowercase) for _ in range(length)])
    while thisName in exclude_list:
        thisName = ''.join([random.choice(string.ascii_lowercase) for _ in range(length)])
    
    return thisName
    
def mergeTrees(tree1, tree2, permitDuplicateNames = True):
    
    # go through the entire dictionary `tree2` and merge each element with the matching element in `tree1`
    
    for (k,v) in tree2.items():
        if tree1.get(k) is None:
            tree1.update([(k, v)])
        else:
            if permitDuplicateNames:
                tree1[k].append(v)
            else:
                # need to rename k
                name = getRandomName(len(k), tree1.keys())
                tree1.update([(name, v)])
    
    return tree1
                
    
    
def getTree(ID, minDepth = 3, maxDepth = 3, 
                minBranch = 2, maxBranch = 3):
    random.seed(ID)
    return getTree_aux(ID, depth = random.randint(minDepth, maxDepth), 
                           branch = random.randint(minBranch, maxBranch))
    
    
    
def getTree_aux(ID, depth, branch):

    if depth == 0:    # inequality isn't correct
        return {}
    
    
    # because we are recursing, we can assume this is the root of the tree:
    name = getRandomName()
    tree = {name : []}   # random name of node? if this is long enough then we *shouldn't* 
                         # get any duplicates, but these can be picked up during the unroll...
    

    
    for b in range(branch):
        t = getTree_aux(ID, depth - 1, branch)
        # need to attach the new tree, rather than just merging it...
        # is it enough to get the root of the branched trees and form those into the list for this tree? 
        tree = mergeTrees(ID, tree, t, permitDuplicateNames = False)
        if len(list(t.keys())) > 0:
            tree[name].append(list(t.keys())[0])
    
    return tree
    
    
def checkCorrect(ID, answer):
    
    tree, node = getTree(ID)
    
    return (bfs([],tree, node) == answer)

The following piece of code can be used to define standard question formats which are then altered to include randomly-generated sections via a placeholder (such as '%') and a function pointer. This is not a perfect implementation, but as a starting point can be useful when varying the text used to define questions.

In [18]:
def question(text, placeholder, func):
    
    # given some text for a question, replace the placeholders with randomly-produced output 
    # from the provided function 
    
    # for each placeholder in text:
    #   replace placeholder with func() output
    new_text = ''
    for word in text.split():
        new_word = word
        if word == placeholder:
            new_word = func()
        
        new_text = new_text + ' ' + new_word
    
    return(new_text.lstrip())
    

    

For example:

In [26]:
print(question('hello, my name is % please call me %', '%', getRandomName))
print(question('hello, my name is % please call me %', '%', getRandomName))
print(question('hello, my name is % please call me %', '%', getRandomName))


hello, my name is grac please call me jkyk
hello, my name is rnfc please call me mtne
hello, my name is tung please call me ztfx


Below is the text and code chunks which are actually generated in the produced PDF. The aim is to have the same chunk of text at the top (question_id_text) which reminds them of the random seed used for to generate the PDF, and then underneath have the question itself. 

In [152]:
seed = random.randrange(1e6)

intro_text = """
# Auto-generated lab sheets

Below is an example of an auto-generated lab sheet for INT1. The code which generates the questions will be hosted 
online and exposed via an API so the students can check their answers before the solutions are released. Provided 
they have their question ID (which is just a random seed), then they can upload their solution along with their ID 
and the server will re-produce the tree from the ID and check it against my unit tests to determine the correct 
answer.

"""

question_id_text = """
Please ensure you make a note of the ID for this set of questions, as it is the only way to access the correct 
solutions. You can test your solution through the online portal available: [http://www.cs.york.ac.uk/~jstovold/INT1/](http://www.cs.york.ac.uk/~jstovold/INT1/).

Question set ID: """ + str(seed)


text = """
# Breadth-First Search
Below is an auto-generated question on Breadth-First Search (BFS). Implement a function in Python that takes as 
arguments a dictionary of state : list(state) pairs (describing the tree) and the root of the tree and produces a 
list of which states are visited in which order when running BFS. For example, with the tree: """

tree_code = """
{ 'A' : ['B','C'], 
  'B' : ['D','E'], 
  'C' : ['F'], 
  'D' : [], 
  'E' : [], 
  'F' : [] 
} 
"""
text2 = """
the function call bfs(tree, 'A') should return:"""

tree_output = "['A', 'B', 'C', 'D', 'E', 'F']"

p = pprint_jhs.PrettyPrinter(width=15)

tree_code2 = """
tree = """ + p.pformat(getTree(seed)) + """ """


newpage_text = """\clearpage{}"""



The next piece of code pulls the text / code chunks from above into a notebook and writes it to file 'test.ipynb' for testing. The 'nbf.write' isn't needed for PDF production, but if you want to produce a jupyter notebook instead then this would be how to do it.

In [159]:
nb['cells'] = [nbf.v4.new_markdown_cell(intro_text),
               nbf.v4.new_markdown_cell(question_id_text),
               nbf.v4.new_markdown_cell(text),
               nbf.v4.new_code_cell(tree_code),
               nbf.v4.new_markdown_cell(text2),
               nbf.v4.new_code_cell(tree_output),
               nbf.v4.new_markdown_cell(newpage_text),
               nbf.v4.new_code_cell(tree_code2) ]
nbf.write(nb, 'test.ipynb')


This notebook is then exported to PDF via LaTeX:

In [154]:
pdf_exporter = PDFExporter()
(body, resources) = pdf_exporter.from_notebook_node(nb, resources={'metadata': {'name': titlePDF, 
                                                                                'date': datePDF , 
                                                                                'author': authorPDF }})


In [155]:
f = open('temp.pdf', 'wb')
f.write(body)
f.close()